In [1]:
import pandas as  pd
import numpy as np
# from google.colab import drive

In [2]:
# drive.mount('/content/drive')

### Male-Female Recognition with CNN

In [3]:
# Import necessary libraries
import os  # Import the os module for file and path operations
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.applications import Xception
from skimage.filters import gabor
from skimage import color
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf
model_vgg16 = tf.keras.applications.vgg16.VGG16()

In [4]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# Assuming your dataset is stored in a folder called 'path'
dataset_path = 'Real'

# Step 1: Load the dataset
def load_dataset(dataset_path):
    images = []
    labels = []

    for filename in os.listdir(dataset_path):
        img_path = os.path.join(dataset_path, filename)
        if os.path.isfile(img_path):  # Check if it's a file
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Read in color

            # Resize the image to (224, 224)
            img = cv2.resize(img, (224, 224))

            images.append(img)

            # Extract gender information (assumed to be the first character after the ID)
            gender = filename.split('__')[1][0]
            labels.append(gender)

    return np.array(images), np.array(labels)

# Load your dataset
images, labels = load_dataset(dataset_path)
# Check information about the loaded dataset
print(f"Total number of images: {len(images)}")
print(f"Total number of unique labels/classes: {len(set(labels))}")
print(f"Labels/classes: {set(labels)}\n")

# Preprocess the images for VGG16
images = preprocess_input(images)

# Print the shape of the images
print("Original Image Shape:", images.shape)

Total number of images: 5950
Total number of unique labels/classes: 2
Labels/classes: {'M', 'F'}

Original Image Shape: (5950, 224, 224, 3)


In [5]:

# Convert labels to numeric format
labels = np.array([0 if label == 'F' else 1 for label in labels])

# Print the first 10 labels for verification
print(labels[:10])

# Step 3: Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check information about the split dataset
print(f"Number of training samples: {len(X_train)}")
print(f"Number of testing samples: {len(X_test)}")
print(f"Number of validation samples: {len(X_val)}")

print(f"\nNumber of unique labels/classes in training set: {len(set(y_train))}")
print(f"Number of unique labels/classes in testing set: {len(set(y_test))}")
print(f"Number of unique labels/classes in validation set: {len(set(y_val))}\n")

[1 1 1 1 1 1 1 1 1 1]
Number of training samples: 4760
Number of testing samples: 595
Number of validation samples: 595

Number of unique labels/classes in training set: 2
Number of unique labels/classes in testing set: 2
Number of unique labels/classes in validation set: 2



#### AlexNet-like model

In [ ]:
# # Evaluate the model on the train set
# test_loss, test_accuracy = model.evaluate(X_train, y_train)
# print(f'Test Loss: {test_loss:.4f}')
# print(f'Test Accuracy: {test_accuracy:.4f}')

In [ ]:
# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print(f'Test Loss: {test_loss:.4f}')
# print(f'Test Accuracy: {test_accuracy:.4f}\n')

#### Feature Extraction

In [6]:
# Step 4: Load pre-trained VGG16 model
model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling=None, classes=2)


# Step 7: Extract features from the deeper layers of the AlexNet model

# Extract features from VGG16
fc_features_train = model_vgg16.predict(X_train)
fc_features_test = model_vgg16.predict(X_test)
fc_features_val = model_vgg16.predict(X_test)

# Flatten the features
fc_features_train_flat = fc_features_train.reshape(fc_features_train.shape[0], -1)
fc_features_test_flat = fc_features_test.reshape(fc_features_test.shape[0], -1)
fc_features_val_flat = fc_features_val.reshape(fc_features_val.shape[0], -1)


# Output: Extracted features
print("VGG16 Features (Training):", fc_features_train_flat.shape)
print("VGG16 Features (Testing):", fc_features_test_flat.shape)
print("VGG16 Features (validation):", fc_features_val_flat.shape)

19/19 [==============================] - 69s 4s/step
VGG16 Features (Training): (4760, 25088)
VGG16 Features (Testing): (595, 25088)
VGG16 Features (validation): (595, 25088)


In [9]:
# from skimage.filters import gabor

# def apply_gabor_filters(image):
#     scales = [1, 2, 4, 8]
#     angles = [0, 45, 90, 135, 180, 225, 270, 315]
#     gabor_features = []

#     for scale in scales:
#         for angle in angles:
#             gabor_filter_real, gabor_filter_imag = gabor(image[:, :, 0], frequency=1/scale, theta=angle)
#             gabor_magnitude = np.sqrt(gabor_filter_real**2 + gabor_filter_imag**2)
#             gabor_features.extend([np.mean(gabor_magnitude), np.std(gabor_magnitude)])

#     return np.array(gabor_features)

# # Apply Gabor filters to the training set
# gabor_features_train = np.array([apply_gabor_filters(img) for img in X_train])
# print("Xtrain Done")

# # Apply Gabor filters to the testing set
# gabor_features_test = np.array([apply_gabor_filters(img) for img in X_test])
# print("Xtest Done")

# # Apply Gabor filters to the validation set
# gabor_features_val = np.array([apply_gabor_filters(img) for img in X_val])
# print("Xval Done")

# # Step 9: Fusion of features in Vector (V)
# fusion_feature_vector_train = np.concatenate((fc_features_train_flat, gabor_features_train), axis=1)
# fusion_feature_vector_test = np.concatenate((fc_features_test_flat, gabor_features_test), axis=1)
# fusion_feature_vector_val = np.concatenate((fc_features_val_flat, gabor_features_val), axis=1)

# # Output: Extract fusion feature vector (V)
# print("Fusion Feature Vector (Training):", fusion_feature_vector_train.shape)
# print("Fusion Feature Vector (Testing):", fusion_feature_vector_test.shape)
# print("Fusion Feature Vector (Validation):", fusion_feature_vector_val.shape)


In [10]:
# Step 9: Fusion of features in Vector (V)
fusion_feature_vector_train = fc_features_train_flat
fusion_feature_vector_test = fc_features_test_flat
fusion_feature_vector_val = fc_features_val_flat

# Output: Extract fusion feature vector (V)
print("Fusion Feature Vector (Training):", fusion_feature_vector_train.shape)
print("Fusion Feature Vector (Testing):", fusion_feature_vector_test.shape)
print("Fusion Feature Vector (Validation):", fusion_feature_vector_val.shape)

Fusion Feature Vector (Training): (4760, 25088)
Fusion Feature Vector (Testing): (595, 25088)
Fusion Feature Vector (Validation): (595, 25088)


Now fusion_feature_vector_train,  fusion_feature_vector_val, and fusion_feature_vector_test can be used to train and evaluate your machine learning model


Classification Model

In [12]:
from keras.callbacks import EarlyStopping

In [13]:
# Step 6: Build a CNN model for classification
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

classification_model = Sequential([
    Dense(128, activation='relu', input_shape=(fusion_feature_vector_train.shape[1],)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification, so use 'sigmoid' activation
])

# Compile the classification model
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
history = classification_model.fit(
    fusion_feature_vector_train, y_train,
    epochs=50, batch_size=32,
    validation_data=(fusion_feature_vector_val, y_val),
    callbacks=[early_stopping]
)

Epoch 1/50


149/149 [==============================] - 3s 16ms/step - loss: 1.1199 - accuracy: 0.7798 - val_loss: 0.6005 - val_accuracy: 0.7748
Epoch 2/50
149/149 [==============================] - 2s 15ms/step - loss: 0.4634 - accuracy: 0.7985 - val_loss: 0.6615 - val_accuracy: 0.7731
Epoch 3/50
149/149 [==============================] - 2s 15ms/step - loss: 0.4210 - accuracy: 0.7998 - val_loss: 0.6221 - val_accuracy: 0.7748
Epoch 4/50
149/149 [==============================] - 2s 15ms/step - loss: 0.4091 - accuracy: 0.7992 - val_loss: 0.6307 - val_accuracy: 0.7748
Epoch 5/50
149/149 [==============================] - 2s 15ms/step - loss: 0.3856 - accuracy: 0.8000 - val_loss: 0.7656 - val_accuracy: 0.7748
Epoch 6/50
149/149 [==============================] - 2s 16ms/step - loss: 0.3481 - accuracy: 0.8006 - val_loss: 0.8119 - val_accuracy: 0.7748


In [14]:
# Evaluate the classification model
loss, accuracy = classification_model.evaluate(fusion_feature_vector_train, y_train)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

149/149 [==============================] - 1s 4ms/step - loss: 0.4100 - accuracy: 0.8029
Test Loss: 0.4100
Test Accuracy: 0.8029


In [15]:
# Evaluate the classification model
loss, accuracy = classification_model.evaluate(fusion_feature_vector_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

19/19 [==============================] - 0s 3ms/step - loss: 0.4425 - accuracy: 0.8034
Test Loss: 0.4425
Test Accuracy: 0.8034


### Keras Tuning

In [16]:
!pip install keras-tuner

  Obtaining dependency information for keras-tuner from https://files.pythonhosted.org/packages/2b/39/21f819fcda657c37519cf817ca1cd03a8a025262aad360876d2a971d38b3/keras_tuner-1.4.6-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/128.9 kB ? eta -:--:--
   --------- ------------------------------ 30.7/128.9 kB 1.4 MB/s eta 0:00:01
   --------------------- ----------------- 71.7/128.9 kB 991.0 kB/s eta 0:00:01
   ---------------------------------- ----- 112.6/128.9 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 128.9/128.9 kB 847.7 kB/s eta 0:00:00


In [31]:
import keras_tuner
from keras import optimizers
from keras.optimizers import Adam
from keras_tuner.tuners import RandomSearch
from keras.models import Model
from keras import callbacks
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense


# Assuming you have fusion_feature_vector_train, y_train, fusion_feature_vector_val, y_val

def build_model(hp):
    # Define the input layer
    input_layer = Input(shape=(fusion_feature_vector_train.shape[1],))

    # Initial hyperparameters
    units = hp.Int('units', min_value=32, max_value=256, step=20)
    dropout_rate = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.01)
    num_hidden_layers = hp.Int('num_hidden_layers', min_value=1, max_value=10, step=1)

    x = input_layer

    # Add hidden layers based on the hyperparameter
    for _ in range(num_hidden_layers):
        x = Dense(units, activation='relu')(x)
        x = Dropout(dropout_rate)(x)

    # Output layer
    output_layer = Dense(1, activation='sigmoid')(x)

    # Build the model
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [32]:

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='keras_tuner_dir',
    project_name='classification_tuner'
)

# Perform the hyperparameter search
tuner.search(fusion_feature_vector_train, y_train,
             epochs=50,
             validation_data=(fusion_feature_vector_val, y_val),
             callbacks=[early_stopping])



Reloading Tuner from keras_tuner_dir\classification_tuner\tuner0.json


In [33]:
# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the final model with the best hyperparameters
final_model = build_model(best_hps)

# Train the final model
final_model.fit(fusion_feature_vector_train, y_train,
                epochs=50,
                validation_data=(fusion_feature_vector_val, y_val),
                callbacks=[early_stopping])

Epoch 1/50
149/149 [==============================] - 5s 24ms/step - loss: 0.8680 - accuracy: 0.7349 - val_loss: 0.5610 - val_accuracy: 0.7748
Epoch 2/50
149/149 [==============================] - 3s 23ms/step - loss: 0.5500 - accuracy: 0.7878 - val_loss: 0.5623 - val_accuracy: 0.7748
Epoch 3/50
149/149 [==============================] - 4s 25ms/step - loss: 0.5081 - accuracy: 0.7916 - val_loss: 0.5671 - val_accuracy: 0.7748
Epoch 4/50
149/149 [==============================] - 3s 23ms/step - loss: 0.4733 - accuracy: 0.7971 - val_loss: 0.5786 - val_accuracy: 0.7748
Epoch 5/50
149/149 [==============================] - 4s 24ms/step - loss: 0.4471 - accuracy: 0.7973 - val_loss: 0.5727 - val_accuracy: 0.7748
Epoch 6/50
149/149 [==============================] - 3s 23ms/step - loss: 0.4369 - accuracy: 0.7973 - val_loss: 0.5790 - val_accuracy: 0.7748


In [34]:
# Evaluate the classification model
loss, accuracy = final_model.evaluate(fusion_feature_vector_train, y_train)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

149/149 [==============================] - 1s 4ms/step - loss: 0.5035 - accuracy: 0.7979
Test Loss: 0.5035
Test Accuracy: 0.7979


In [35]:
# Evaluate the classification model
loss, accuracy = final_model.evaluate(fusion_feature_vector_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

19/19 [==============================] - 0s 4ms/step - loss: 0.5107 - accuracy: 0.7966
Test Loss: 0.5107
Test Accuracy: 0.7966


### Testing the model on other images 

In [78]:
c = 0
for i in labels:
    if i == 0:
        c = c+1
print("Females:", c)
print("Males:", len(labels)-c)


Females: 1217
Males: 4733


### Exporting Models For Deployment

In [80]:
classification_model.save("saved_model.h5")